In [ ]:
import brightway2 as bw
import bw2regional as bwr
from bw2regional.pandarus_remote import remote, AlreadyExists
import bw2_aware
import rower
import os

In [ ]:
bw.projects.set_current("DF69 Case Study")

# Import ecoinvent 3.5

In [ ]:
bw.create_default_biosphere3()

In [ ]:
bw.create_core_migrations()

In [ ]:
ei = bw.SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA/Ecoinvent/3.5/cutoff/datasets",
    "ecoinvent 3.5 cutoff"
)
ei.apply_strategies()
ei.statistics()

In [ ]:
ei.write_database()

# Import case study inventory

In [ ]:
imp = bw.ExcelImporter("rice_inventory.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location', 'reference product'))
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location'))
imp.match_database(fields=['name', 'location'])
imp.statistics()

In [ ]:
imp.write_database()

# Fix geographic data in LCI databases

## Install basic world definitions

In [ ]:
bwr.bw2regionalsetup()

## Define rest of world locations in ecoinvent

In [ ]:
rower_obj = rower.Rower('ecoinvent 3.5 cutoff')
rower_obj.load_existing(rower_obj.EI_3_5_CUTOFF)
rower_obj.label_RoWs()

## Label locations with their "geocollection"

Need to turn e.g. `"Europe without CH"` into `("ecoinvent", "Europe without CH")`.

In [ ]:
bwr.label_activity_geocollections("ecoinvent 3.5 cutoff")
bwr.label_activity_geocollections("AWARE Case Study")

In [ ]:
bw.databases['ecoinvent 3.5 cutoff']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['AWARE Case Study']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['biosphere3']['geocollections'] = []
bw.databases.flush()

# Import AWARE

In [ ]:
bw2_aware.import_aware()

# Add extension tables

## Define where rice is grown

Our rice inventory is country-specific, but this is too general for China and the US. We use data from Ramankutty on global rice production. Here is the map for part of Asia:

<img src="images/rice-asia.png">

In [ ]:
fp = os.path.abspath("rice.tiff")
assert os.path.isfile(fp)

bwr.geocollections['rice'] = {
    'filepath': fp,
    'band': 1
}

## Add population density

We assume that the spatial pattern of other water-consuming activities follows population density.

In [ ]:
fp = os.path.abspath("gdpweighted.tif")
assert os.path.isfile(fp)

bwr.geocollections['weighted-pop-density'] = {
    'filepath': fp,
    'band': 1
}

## Calculate raster statistics for our extension tables

In [ ]:
crop_rasters = [
    'rice',
    'weighted-pop-density',
]

for raster in crop_rasters:
    try:
        remote.calculate_rasterstats('world-topo-watersheds-aware', raster)
    except AlreadyExists:
        pass

In [ ]:
for raster in crop_rasters:
    remote.rasterstats_as_xt('world-topo-watersheds-aware', raster, "xt-" + raster)